In [ ]:
# Finds the path between a source ASN and a destination ASN 
import pybgpstream

import networkx as nx

import matplotlib.pyplot as plt
import pandas as pd
import csv

# Convert ASes on AS_Path attribute to list
def as_path_to_list(as_path):
    
    as_list = []

    for asn in as_path:
        as_list.append(int(asn))

    return(as_list)

def add_edge_to_graph(graph, as_path):
    path_len = len(as_path)
    for k,asn in enumerate(as_path):
        if (k+1) is not path_len:
            graph.add_edge(as_path[k],as_path[k+1])

# Find paths from MS ASN to route collectors
def as_path_ms_rc(graph):
    stream = pybgpstream.BGPStream(
        from_time="2024-02-14 00:00:01 CET",
        until_time="2024-03-14 23:59:00 CET",
        collectors=["rrc00", "rrc03", "rrc25", "route-views.amsix" ],
        record_type="ribs",     
        filter = 'prefix less 52.101.0.0/16 and path _8075' 
       )
    stream.set_data_interface_option("broker", "cache-dir", "/home/shyam/jupy/cache")
    
    # Find paths from MS ASN to route collectors
    for rec in stream.records():
        for elem in rec:
            #print(elem)
            as_path1 = as_path_to_list(elem.fields["as-path"].split())
            #print("AS path from MS to RC is ", as_path1)
            add_edge_to_graph(graph, as_path1)
    return graph

# Find paths from ING ASN to route collectors
def as_path_ing_rc(graph):
    stream = pybgpstream.BGPStream(
        from_time="2024-02-14 00:00:01 CET",
        until_time="2024-03-14 23:59:00 CET",
        collectors=["rrc00", "rrc03", "rrc25", "route-views.amsix"],
        record_type="ribs",     
            filter = 'path _56517' 
       )
    stream.set_data_interface_option("broker", "cache-dir", "/home/shyam/jupy/cache")
    
    # Find paths from MS ASN to route collectors
    for rec in stream.records():
        for elem in rec:
            #print(elem)
            as_path2 = as_path_to_list(elem.fields["as-path"].split())
            #print("AS path from ING to RC is ", as_path2)
            add_edge_to_graph(graph, as_path2)
    return graph

In [ ]:
# Change source and destination as per requirments here
source = "15916" 
dest = "8075"
graph = nx.Graph()
graph1 = as_path_ms_rc(graph)



In [ ]:
print(graph1)

In [ ]:
# Dump MS annoucement into a graph object
import pickle

# Dump graph object into pickle form
with open('/home/shyam/phd-work/datasets/critical_infra_nl/pickle_file_path_ms_announcement_02_14_03_14_collector_limit_routeviews.p', 'wb') as pickleFile:
    pickle.dump(graph1, pickleFile)

In [ ]:
print(graph1)

In [1]:
import pickle
# Unpickle a pickle file to get a Graph object
with open('/home/shyam/phd-work/datasets/critical_infra_nl/pickle_file_path_ms_announcement_02_14_03_14_collector_limit_routeviews.p', 'rb') as pickleFile:
    graph_ms = pickle.load(pickleFile)
print(graph_ms)

Graph with 174 nodes and 197 edges


In [ ]:
# Merge a graph obtained from MS announcement to new graph
graph2 = as_path_ing_rc(graph_ms)

In [ ]:
print(graph2)

In [ ]:
# Dump MS annoucement into a graph object
import pickle

# Dump graph object into pickle form
with open('/home/shyam/phd-work/datasets/critical_infra_nl/pickle_file_path_abn_ms_02_14_03_14_collector_limit_routeviews.p', 'wb') as pickleFile:
    pickle.dump(graph2, pickleFile)

In [2]:
import pickle
# Unpickle a pickle file to get a Graph object
with open('/home/shyam/phd-work/datasets/critical_infra_nl/pickle_file_path_abn_ms_02_14_03_14_collector_limit_routeviews.p', 'rb') as pickleFile:
    graph_new = pickle.load(pickleFile)
print(graph_new)

Graph with 201 nodes and 360 edges


In [3]:
import networkx as nx
import pandas as pd
source = "15916"
dest = "8075"

# Get all the paths with cutoff = 5 (No. of nodes between a source and a destination)
paths = nx.all_simple_paths(graph_new, int(source), int(dest), cutoff=5)


# Save all the paths into a file    
df = pd.DataFrame(paths)
df.to_csv('/home/shyam/phd-work/datasets/critical_infra_nl/path_ms_abn_02_14_03_14_cutoff_05_collector_limit_routviews.csv', index=False, header=False)    


In [ ]:
print(df)

In [ ]:
print(df)

In [ ]:
import pandas as pd
df = pd.read_csv("/home/shyam/phd-work/datasets/critical_infra_nl/path_ms_eneco_02_14_18_cutoff_08_collector_limit.csv")

print(len(df))
#print(df[df["1273"] == 702])